# 365 Data Preprocessing
### Samuel Mendez
The raw data provided in this project has several improvements that can be made to make data analysis cleaner. This journal contains three functions that help clean the data while allowing some user input. Each page visit has it's unique session id for each user id. The data scientist can select the sessions that seems most important for their analysis. For example, the data scientist can specify to only see the last three sessions. They can also specify the pages they want removed from the clean data set.  
The following functions are below:  
`remove_duplicate_pages`: removes consequtively occuring pages from each row.  
`group_user`: since there are multiple session for the same users, this function consolidates that data into one row and removes the session column since it is not essantial for our data analysis purposes.  
`remove_pages`: this function allows for the user to specify what pages they want to include in their clean data analysis.  
Since subscription type can be an essential column for finding patterns in the data, I decided to keep the column and let users select the subscription type in the next notebook for data analysis.

In [1]:
# import libraries
import pandas as pd
import numpy as np

In [2]:
# import data
raw_data = pd.read_csv("user_journey_raw.csv")

### Data Exploration

In [3]:
raw_data.shape

(9935, 4)

In [4]:
# view first 25 rows of data
raw_data.head(25)

,user_id,session_id,subscription_type,user_journey
0,1516,2980231,Annual,Homepage-Log in-Log in-Log in-Log in-Log in-Lo...
1,1516,2980248,Annual,Other-Sign up-Sign up-Sign up-Sign up-Sign up-...
2,1516,2992252,Annual,Log in-Log in-Log in-Log in-Log in-Log in
3,1516,3070491,Annual,Homepage-Log in-Log in-Log in-Log in-Log in-Lo...
4,1516,3709807,Annual,Log in-Log in-Log in-Log in-Log in-Log in-Log ...
5,1516,3723132,Annual,Checkout-Checkout-Checkout-Checkout
6,1516,3723365,Annual,Checkout-Checkout-Checkout-Checkout-Checkout-C...
7,1516,3723382,Annual,Checkout-Checkout-Checkout-Checkout-Checkout-C...
8,1516,3723427,Annual,Checkout-Checkout
9,1516,3723483,Annual,Coupon-Coupon


In [5]:
# check missing values
raw_data.isnull().sum()

user_id              0
session_id           0
subscription_type    0
user_journey         0
dtype: int64

### Data Manipulation/Preprocessing
Since we dont care about the time a person spent on any webpage or the number of times they clicked on a particular page, we can remove duplicate sequntial page visits and keep the sequence that lead to the subscription. We can also consolidate the `session_id` for each user into one `user_id`.

In [6]:
# define function to remove consecutively repeating pages
def remove_duplicate_pages(df, desired_column):
    # define function to process the string in each column
    def string_clean(page_seq):
        # split each string at the hyphen
        pages = page_seq.split('-')
        # create empty list to store the new string
        new_seq = []
        prev_page = None
        
        # iterate through the string to find + remove consecutive pages
        for page in pages:
            if page != prev_page:
                new_seq.append(page)
                prev_page = page
                
        return '-'.join(new_seq)
    new_df = df.copy()
    
    # apply string_clean to the entire df
    new_df[desired_column] = new_df[desired_column].apply(string_clean)
    
    return new_df
 
        
    

In [7]:
# create a function to consolidate same user_id data into one row
def group_users(df, group_name = 'user_id', target_column = 'user_journey', sessions = 'All', count_from = 'first',secondary_group='subscription_type',):
    # make a copy of df to not alter the original
    grouped_df = df.copy()
    
    # group data based on sessions specified by user
    if sessions != 'All':
        if count_from == 'first':
            grouped_df = grouped_df.groupby(group_name).head(sessions)
        elif count_from == 'last':
            grouped_df = grouped_df.groupby(group_name).tail(sessions)
    
    # create one string for user_journer from specified sessions
    grouped_df = grouped_df.groupby([group_name, secondary_group], as_index=False)[target_column].agg(lambda x:'-'.join(x))
    
    return grouped_df

    

In [8]:
grouped_users = group_users(raw_data)

In [9]:
grouped_users

,user_id,subscription_type,user_journey
0,1516,Annual,Homepage-Log in-Log in-Log in-Log in-Log in-Lo...
1,3395,Annual,Other-Other-Pricing-Sign up-Sign up-Log in-Log...
2,10107,Annual,Homepage-Homepage-Homepage-Homepage-Homepage-H...
3,11145,Monthly,Homepage-Log in-Log in-Log in-Log in-Log in-Ho...
4,12400,Monthly,Homepage-Career tracks-Career tracks-Career tr...
...,...,...,...
1345,509060,Annual,Other-Other-Other-Other-Other-Other-Other-Othe...
1346,509061,Annual,Coupon-Coupon-Coupon-Coupon-Coupon-Coupon-Coup...
1347,509085,Annual,Coupon-Coupon-Coupon-Coupon-Coupon-Coupon-Coup...
1348,509095,Annual,Other-Other-Other-Other-Other-Other-Other-Othe...


In [10]:
data_no_duplicates = remove_duplicate_pages(grouped_users, 'user_journey')

In [11]:
data_no_duplicates.head(10)

,user_id,subscription_type,user_journey
0,1516,Annual,Homepage-Log in-Other-Sign up-Log in-Homepage-...
1,3395,Annual,Other-Pricing-Sign up-Log in-Homepage-Pricing-...
2,10107,Annual,Homepage-Career tracks-Homepage-Career tracks-...
3,11145,Monthly,Homepage-Log in-Homepage-Log in-Homepage-Log i...
4,12400,Monthly,Homepage-Career tracks-Sign up-Log in-Other-Ca...
5,13082,Monthly,Checkout-Homepage-Sign up-Log in-Checkout
6,14415,Monthly,Pricing-Sign up-Pricing-Sign up-Homepage-Log i...
7,15630,Annual,Log in-Checkout-Pricing-Checkout-Other-Homepag...
8,16589,Quarterly,Homepage-Career tracks-Homepage-Career tracks-...
9,19458,Annual,Homepage-Sign up-Log in-Other-Homepage-Pricing...


In [12]:
# create a function that removes unessential page visits
def remove_pages(df, pages_list, target_column = 'user_journey'):
    # make copy of df
    new_df = df.copy()
    
    # create function to check page visits for each row and remove them if they are in the list provided
    def check_pages(all_pages):
        # split each page in the string for each row
        pages = all_pages.split('-')
        # create an empty list to store pages that are not in the specified list
        new_pages = []
        
        # add words to new_pages that are not in the specified list
        for page in pages:
            if page not in pages_list:
                new_pages.append(page)
        # join the pages into one string seperated by a hyphen
        return '-'.join(new_pages)

    # apply check_pages funtion to entire target column
    new_df[target_column] = new_df[target_column].apply(check_pages)
    
    return new_df

    

In [14]:
pages = ['Other']

clean_data = remove_pages(data_no_duplicates, pages)

In [15]:
clean_data.head(10)

,user_id,subscription_type,user_journey
0,1516,Annual,Homepage-Log in-Sign up-Log in-Homepage-Log in...
1,3395,Annual,Pricing-Sign up-Log in-Homepage-Pricing-Checkout
2,10107,Annual,Homepage-Career tracks-Homepage-Career tracks-...
3,11145,Monthly,Homepage-Log in-Homepage-Log in-Homepage-Log i...
4,12400,Monthly,Homepage-Career tracks-Sign up-Log in-Career t...
5,13082,Monthly,Checkout-Homepage-Sign up-Log in-Checkout
6,14415,Monthly,Pricing-Sign up-Pricing-Sign up-Homepage-Log i...
7,15630,Annual,Log in-Checkout-Pricing-Checkout-Homepage-Log ...
8,16589,Quarterly,Homepage-Career tracks-Homepage-Career tracks-...
9,19458,Annual,Homepage-Sign up-Log in-Homepage-Pricing-Homep...


In [ ]:
# export clean data to new csv file
clean_data.to_csv(r'C:\Users\mende\User Journey Project Py\clean_data.csv', index = False)

End of journal.